In [6]:
import pandas as pd

vinhoBranco = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=';')

vinhoTinto = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')

In [7]:
import seaborn as sns
correlacao = vinhoTinto.corr()
sns.heatmap(correlacao,xticklabels=correlacao.columns.values, yticklabels=correlacao.columns.values)


<AxesSubplot:>

In [8]:
#criando uma coluna chamada 'type' e colocando 0 quando branco e 1 quando tinto
vinhoTinto['type']=1
vinhoBranco['type']=0
#vamos unir os dataset em um novo dataset chamado vinhos
vinhos = vinhoTinto.append(vinhoBranco,ignore_index=True)

vinhos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [9]:
#vamos utilizar um modelo sequencial - rede neural - do keras
from tensorflow.keras.models import Sequential
#vamos importar o Dense para ser o modelo de ativação entre camadas
from keras.layers import Dense

#instanciar o modelo
rede = Sequential()

#Criar a camada de entrada
rede.add(Dense(12,activation='relu',input_shape=(11,)))

#Criando a camada oculta (montada automaticamente pela rede neural)
rede.add(Dense(8,activation='relu'))

#Criando a camada de saída (responde a classificação pela rede neural)
rede.add(Dense(1,activation='relu'))

#compilar a rede neural
rede.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
import numpy as np #módulo para trabalhar com números
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#definir o dataset de treino e de teste
caracteristicas = vinhos.iloc[:,0:11]
classes = np.ravel(vinhos.type)

carac_treino,carac_teste, classes_treino, classes_teste = train_test_split(caracteristicas,classes,test_size=0.25,random_state=5)

#Padronizar os valores do treino
padrao = StandardScaler().fit(carac_treino)
carac_treino = padrao.transform(carac_treino)
carac_teste = padrao.transform(carac_teste)


In [11]:
rede.fit(carac_treino,classes_treino,epochs=20,batch_size=1,verbose=1)

Epoch 1/20
4872/4872 [==============================] - 7s 1ms/step - loss: 0.5137 - accuracy: 0.9339
Epoch 2/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0636 - accuracy: 0.9945
Epoch 3/20
4872/4872 [==============================] - 7s 1ms/step - loss: 0.0433 - accuracy: 0.9956
Epoch 4/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0708 - accuracy: 0.9951
Epoch 5/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0546 - accuracy: 0.9961
Epoch 6/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0462 - accuracy: 0.9963
Epoch 7/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0262 - accuracy: 0.9970
Epoch 8/20
4872/4872 [==============================] - 5s 1ms/step - loss: 0.0382 - accuracy: 0.9973
Epoch 9/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.0421 - accuracy: 0.9972
Epoch 10/20
4872/4872 [==============================] - 6s 1ms/step - loss: 0.054

In [12]:
#avaliando a rede treinada com os dados de teste
resultados = rede.evaluate(carac_teste,classes_teste)

print('%s: %.2f%%' %(rede.metrics_names[1],resultados[1]*100))



51/51 [==============================] - 0s 959us/step - loss: 0.0766 - accuracy: 0.9951
accuracy: 99.51%


In [13]:
#como testar a rede neural
def mostraTipoVinho(valor):
  print(valor[0])
  if valor[0]>0:
    return 'Vinho Tinto'
  else:
    return 'Vinho Branco'

tipo = rede.predict(vinhos.iloc[495:496,0:11])
print(vinhos.loc[496])
print(mostraTipoVinho(tipo),tipo)

fixed acidity            7.8000
volatile acidity         0.5200
citric acid              0.2500
residual sugar           1.9000
chlorides                0.0810
free sulfur dioxide     14.0000
total sulfur dioxide    38.0000
density                  0.9984
pH                       3.4300
sulphates                0.6500
alcohol                  9.0000
quality                  6.0000
type                     1.0000
Name: 496, dtype: float64
[0.]
Vinho Branco [[0.]]
